In [2]:
import os
import sys
import re
import pandas as pd

from tqdm import tqdm
from typing import Any
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root not in sys.path:
    sys.path.append(root)

from src.model import APIServer

INFO 05-23 07:56:57 [__init__.py:239] Automatically detected platform cuda.


In [3]:
def process(
    data: str,
    target: str,
    target_values: list[str],
) -> list[str]:
    target_values = " or ".join(target_values)

    def build_meta_prompt(X_data: Any) -> str:
        return f"""Based on the input data, predict the value of '{target}':

{X_data}

**IMPORTANT: Respond with ONLY {target_values}. Do not include any explanations, reasoning, or additional text."
"""

    return list(map(build_meta_prompt, data))

In [4]:
def parse(text: str) -> str:
    match = re.search(r"(DEAD|ALIVE)", text)
    if match and len(match.groups()) == 1:
        return match.group(1)
    else:
        return "ERROR"

In [ ]:
class TitanicDataset(Dataset):
    def __init__(
        self,
        path: str,
        train: bool = True,
        shuffle: bool = True,
        train_size: float = 0.8,
    ):
        super().__init__()
        self.target = "Survived"
        self.target_values = ["DEAD", "ALIVE"]

        df = pd.read_csv(path)

        assert (
            self.target in df.columns
        ), f"Target column '{self.target}' not found in the dataframe."

        self.feature_labels = df.drop(columns=[self.target]).columns.tolist()

        X = df.drop(columns=[self.target])
        y = df[self.target]

        X_train, X_test, y_train, y_test = train_test_split(
            X,
            y,
            train_size=train_size,
            random_state=42,
            shuffle=shuffle,
            stratify=y,
        )

        self.X = X_train if train else X_test
        self.y = y_train if train else y_test

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_dict = str(self.X.iloc[idx].to_dict())
        y_value = self.target_values[self.y.iloc[idx].item()]
        return X_dict, y_value

In [7]:
dataset = TitanicDataset("../dataset/titanic-dataset.csv", train=False)
data_loader = DataLoader(dataset, batch_size=4, shuffle=False)
api_server = APIServer("localhost", port=23456)
num_epochs = 5

total_accuracy = []
total_error_rate = []

for epoch in range(num_epochs):
    corrects = 0
    errors = 0
    for X, y in tqdm(data_loader):
        prompts = process(X, "Survived", ["DEAD", "ALIVE"])
        responses = api_server.request(prompts, {"n": 1})
        y_preds = [parse(response) for response in responses]
        errors += sum([1 if preds == "ERROR" else 0 for preds in y_preds])
        corrects += sum([1 if a == b else 0 for a, b in zip(y_preds, y)])

    total_accuracy.append(corrects / len(data_loader.dataset))
    total_error_rate.append(errors / len(data_loader.dataset))

print("Total Accuracy:", total_accuracy)
print("Total Error Rate:", total_error_rate)

print(f"Average Accuracy: {sum(total_accuracy) / num_epochs}")
print(f"Average Error Rate: {sum(total_error_rate) / num_epochs}")

  0%|          | 0/45 [00:00<?, ?it/s]

100%|██████████| 45/45 [00:51<00:00,  1.14s/it]

Total Accuracy: [0.6145251396648045, 0.6703910614525139, 0.664804469273743, 0.6256983240223464, 0.6536312849162011]
Total Error Rate: [0.12290502793296089, 0.055865921787709494, 0.09497206703910614, 0.0782122905027933, 0.11173184357541899]
Average Accuracy: 0.6458100558659218
Average Error Rate: 0.09273743016759776
